# Calibration of the deflection of a tube

## Create a calibration problem

In [1]:
import openturns as ot
ot.RandomGenerator.SetSeed(0)

We use the variable names `De` for the external diameter and `di` for the internal diameter because the symbolic function engine is not case sensitive, hence the variable names `D` and `d` would not be distiguished.

In [2]:
inputsvars=["F","L","a","De","di","E"]
formula = "var I:=pi_*(De^4-di^4)/32; var b:=L-a; g1:=-F*a^2*(L-a)^2/(3*E*L*I); g2:=-F*b*(L^2-b^2)/(6*E*L*I)*1.e6; g3:=F*a*(L^2-a^2)/(6*E*L*I)*1.e6"
g = ot.SymbolicFunction(inputsvars,["g1","g2","g3"],formula)
g.setOutputDescription(["Deflection","Left angle","Right angle"])

In [3]:
XF=ot.Normal(1,0.1)
XE=ot.Normal(2e5,2e3)
XF.setDescription(["Force"])
XE.setDescription(["Young Modulus"])

In [4]:
XL = ot.Dirac(1.5)
Xa = ot.Dirac(1.0)
XD = ot.Dirac(0.8)
Xd = ot.Dirac(0.1)
XL.setDescription(["Longueur"])
Xa.setDescription(["Location"])
XD.setDescription(["External diameter"])
Xd.setDescription(["Internal diameter"])

In [5]:
inputDistribution = ot.ComposedDistribution([XF,XL,Xa,XD,Xd,XE])

In [6]:
size = 10
inputSample = inputDistribution.getSample(size)
inputSample[0:5]

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=6 description=[Force,Longueur,Location,External diameter,Internal diameter,Young Modulus] data=[[1.06082,1.5,1,0.8,0.1,199059],[0.873383,1.5,1,0.8,0.1,200522],[0.956173,1.5,1,0.8,0.1,195420],[1.12055,1.5,1,0.8,0.1,197434],[0.781861,1.5,1,0.8,0.1,197376]]

In [7]:
outputDeflection = g(inputSample)
outputDeflection[0:5]

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=3 description=[Deflection,Left angle,Right angle] data=[[-7.36434e-06,-14.7287,18.4108],[-6.01889e-06,-12.0378,15.0472],[-6.76148e-06,-13.523,16.9037],[-7.84299e-06,-15.686,19.6075],[-5.47404e-06,-10.9481,13.6851]]

In [8]:
observationNoiseSigma = [0.1e-6,1.5,1.5]
observationNoiseCovariance = ot.CovarianceMatrix(3)
for i in range(3):
    observationNoiseCovariance[i,i] = observationNoiseSigma[i]**2

In [9]:
noiseSigma = ot.Normal([0.,0.,0.],observationNoiseCovariance)
sampleObservationNoise = noiseSigma.getSample(size)
outputObservations = outputDeflection + sampleObservationNoise
outputObservations[0:5]

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=3 description=[Deflection,Left angle,Right angle] data=[[-7.33205e-06,-14.06,16.8537],[-6.10456e-06,-11.3273,14.859],[-6.72634e-06,-10.8494,17.009],[-7.92113e-06,-16.7683,19.2456],[-5.65284e-06,-10.346,15.7368]]

In [10]:
inputObservations = ot.Sample(size,2)
inputObservations[:,0] = inputSample[:,0] # F
inputObservations[:,1] = inputSample[:,5] # E
inputObservations.setDescription(["Force","Young Modulus"])
inputObservations[0:5]

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=2 description=[Force,Young Modulus] data=[[1.06082,199059],[0.873383,200522],[0.956173,195420],[1.12055,197434],[0.781861,197376]]

## Setting up the calibration

In [11]:
XL = 1.4 # Exact : 1.5
Xa = 1.2 # Exact : 1.0
XD = 0.7 # Exact : 0.8
Xd = 0.2 # Exact : 0.1
candidate = ot.Point([XL,Xa,XD,Xd])

In [12]:
sigmaXL = 0.1 * XL
sigmaXa = 0.1 * Xa
sigmaXD = 0.1 * XD
sigmaXd = 0.1 * Xd
parameterCovariance = ot.CovarianceMatrix(4)
parameterCovariance[0,0] = sigmaXL**2
parameterCovariance[1,1] = sigmaXa**2
parameterCovariance[2,2] = sigmaXD**2
parameterCovariance[3,3] = sigmaXd**2
parameterCovariance

class=CovarianceMatrix dimension=4 implementation=class=MatrixImplementation name=Unnamed rows=4 columns=4 values=[0.0196,0,0,0,0,0.0144,0,0,0,0,0.0049,0,0,0,0,0.0004]

In [13]:
calibratedIndices = [1,2,3,4]
model = ot.ParametricFunction(g, calibratedIndices, candidate)

In [14]:
sigmaObservation = [0.2e-6,1.3,1.3] # Exact : [0.1e-6,1.5,1.5]

In [15]:
errorCovariance = ot.CovarianceMatrix(3)
errorCovariance[0,0] = sigmaObservation[0]**2
errorCovariance[1,1] = sigmaObservation[1]**2
errorCovariance[2,2] = sigmaObservation[2]**2

## See the output error dispersion

In [16]:
import pylab as pl
import openturns.viewer
outputDimension = outputObservations.getDimension()
fig = pl.figure(figsize=(16, 4))
for i in range(outputDimension):
    ax = fig.add_subplot(1, outputDimension, i+1)
    graph = ot.HistogramFactory().build(outputObservations[:,i]).drawPDF()    
    _ = ot.viewer.View(graph, figure=fig, axes=[ax])

## Calibration with gaussian linear least squares

In [17]:
algo = ot.GaussianLinearCalibration(model, inputObservations, outputObservations, candidate, parameterCovariance, errorCovariance)

In [18]:
algo.run()

In [19]:
calibrationResult = algo.getResult()

## Analysis of the results

In [20]:
thetaMAP = calibrationResult.getParameterMAP()
thetaMAP

class=Point name=Unnamed dimension=4 values=[1.52849,1.04743,0.892749,0.199633]

In [21]:
thetaPosterior = calibrationResult.getParameterPosterior()
thetaPosterior.getCovariance()

class=CovarianceMatrix dimension=4 implementation=class=MatrixImplementation name=Unnamed rows=4 columns=4 values=[0.00768496,0.00780418,0.00107517,-2.04709e-06,0.00780418,0.00807009,0.000925504,-1.76213e-06,0.00107517,0.000925504,0.000362111,8.64e-06,-2.04709e-06,-1.76213e-06,8.64e-06,0.000399984]

## Par résolution d'un problème de moindres carrés linéaires

In [22]:
model.setParameter(candidate)

In [23]:
modelObservations = model(inputObservations)
modelObservations[0:5]

class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=5 dimension=3 description=[Deflection,Left angle,Right angle] data=[[-3.12137e-06,-10.4046,16.9074],[-2.5511e-06,-8.50366,13.8185],[-2.86585e-06,-9.55282,15.5233],[-3.32424e-06,-11.0808,18.0063],[-2.32017e-06,-7.73389,12.5676]]

In [24]:
residuals = modelObservations - outputObservations
outputDimension = modelObservations.getDimension()
# Stack the residuals by observation, 
#   deltay = [residuals[i,0],residuals[i,1],...,residuals[i,outputDimension]]
deltay = ot.Point(size*outputDimension)
for i in range(size):
    for j in range(outputDimension):
        deltay[i*outputDimension + j] = residuals[i,j]
deltay

class=Point name=Unnamed dimension=30 values=[4.21068e-06,3.65544,0.0536782,3.55346e-06,2.82368,-1.04051,3.86049e-06,1.2966,-1.48567,4.59688e-06,5.68747,-1.23932,3.33267e-06,2.61215,-3.16927,4.02355e-06,3.09053,-1.39586,3.74777e-06,3.42449,-1.95866,4.60791e-06,5.22109,-0.486334,4.30206e-06,1.45748,-0.526577,4.092e-06,4.28115,-2.70137]

In [25]:
B = parameterCovariance
LB = B.computeCholesky()
LB

class=TriangularMatrix dimension=4 implementation=class=MatrixImplementation name=Unnamed rows=4 columns=4 values=[0.14,0,0,0,0,0.12,0,0,0,0,0.07,0,0,0,0,0.02]

In [26]:
errorCovariance

class=CovarianceMatrix dimension=3 implementation=class=MatrixImplementation name=Unnamed rows=3 columns=3 values=[4e-14,0,0,0,1.69,0,0,0,1.69]

In [27]:
dimension = errorCovariance.getDimension()
R = ot.CovarianceMatrix(deltay.getSize())
for i in range(size):
    for j in range(dimension):
        for k in range(dimension):
            R[i * dimension + j, i * dimension + k] = errorCovariance[j,k]
R[0:10,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=10 columns=30 values=[4e-14,0,0,0,0,0,0,0,0,0,0,1.69,0,0,0,0,0,0,0,0,0,0,1.69,0,0,0,0,0,0,0,0,0,0,4e-14,0,0,0,0,0,0,0,0,0,0,1.69,0,0,0,0,0,0,0,0,0,0,1.69,0,0,0,0,0,0,0,0,0,0,4e-14,0,0,0,0,0,0,0,0,0,0,1.69,0,0,0,0,0,0,0,0,0,0,1.69,0,0,0,0,0,0,0,0,0,0,4e-14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [28]:
LR = R.computeCholesky()
LR[0:10,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=10 columns=30 values=[2e-07,0,0,0,0,0,0,0,0,0,0,1.3,0,0,0,0,0,0,0,0,0,0,1.3,0,0,0,0,0,0,0,0,0,0,2e-07,0,0,0,0,0,0,0,0,0,0,1.3,0,0,0,0,0,0,0,0,0,0,1.3,0,0,0,0,0,0,0,0,0,0,2e-07,0,0,0,0,0,0,0,0,0,0,1.3,0,0,0,0,0,0,0,0,0,0,1.3,0,0,0,0,0,0,0,0,0,0,2e-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [29]:
parameterDimension = candidate.getDimension()
ILB = ot.IdentityMatrix(parameterDimension)
invLB = LB.solveLinearSystem(ILB)
invLB[0:5,:]

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=4 columns=4 values=[7.14286,0,0,0,0,8.33333,0,0,0,0,14.2857,0,0,0,0,50]

In [30]:
transposedGradientObservations = ot.Matrix(parameterDimension,size*outputDimension)
for i in range(size):
    g = model.parameterGradient(inputObservations[i])
    print(i, g)
    for j in range(outputDimension):
        for k in range(parameterDimension):
            transposedGradientObservations[k,i*outputDimension + j] = g[k,j]
transposedGradientObservations

0 [[  -2.89841e-05 -57.5967       78.9632      ]
 [   2.60114e-05  49.8552      -63.9447      ]
 [   1.7956e-05   59.8535      -97.2619      ]
 [  -4.188e-07    -1.396         2.2685      ]]
1 [[  -2.36888e-05 -47.0739       64.5367      ]
 [   2.12592e-05  40.7467      -52.2621      ]
 [   1.46755e-05  48.9184      -79.4923      ]
 [  -3.42286e-07  -1.14095       1.85405     ]]
2 [[  -2.66114e-05 -52.8817       72.4991      ]
 [   2.38821e-05  45.7739      -58.71        ]
 [   1.64861e-05  54.9537      -89.2998      ]
 [  -3.84516e-07  -1.28172       2.0828      ]]
3 [[   -3.0868e-05   -61.3402        84.0955      ]
 [    2.7702e-05    53.0956       -68.1008      ]
 [    1.91231e-05   63.7437      -103.584       ]
 [   -4.4602e-07    -1.48673        2.41594     ]]
4 [[  -2.15444e-05 -42.8126       58.6947      ]
 [   1.93347e-05  37.0582      -47.5312      ]
 [   1.3347e-05   44.4902      -72.2965      ]
 [  -3.11301e-07  -1.03767       1.68622     ]]
5 [[  -2.81713e-05 -55.9814      

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=4 columns=30 values=[-2.89841e-05,2.60114e-05,1.7956e-05,-4.188e-07,-57.5967,49.8552,59.8535,-1.396,78.9632,-63.9447,-97.2619,2.2685,-2.36888e-05,2.12592e-05,1.46755e-05,-3.42286e-07,-47.0739,40.7467,48.9184,-1.14095,64.5367,-52.2621,-79.4923,1.85405,-2.66114e-05,2.38821e-05,1.64861e-05,-3.84516e-07,-52.8817,45.7739,54.9537,-1.28172,72.4991,-58.71,-89.2998,2.0828,-3.0868e-05,2.7702e-05,1.91231e-05,-4.4602e-07,-61.3402,53.0956,63.7437,-1.48673,84.0955,-68.1008,-103.584,2.41594,-2.15444e-05,1.93347e-05,1.3347e-05,-3.11301e-07,-42.8126,37.0582,44.4902,-1.03767,58.6947,-47.5312,-72.2965,1.68622,-2.81713e-05,2.52819e-05,1.74525e-05,-4.07055e-07,-55.9814,48.457,58.1749,-1.35685,76.7487,-62.1514,-94.5343,2.20488,-2.59698e-05,2.33062e-05,1.60886e-05,-3.75245e-07,-51.6067,44.6703,53.6288,-1.25082,70.7511,-57.2945,-87.1468,2.03258,-3.11457e-05,2.79513e-05,1.92952e-05,-4.50033e-07,-61.8921,53.5732,64.3172,-1.50011,84.852,-68.7135,-104.515,2.43768,-2.9564e-05,2.65318e-05,1.83153e-05,-4.27178e-07,-58.7489,50.8525,61.0508,-1.42393,80.5428,-65.2239,-99.2076,2.31388,-2.92205e-05,2.62236e-05,1.81025e-05,-4.22216e-07,-58.0665,50.2618,60.3417,-1.40739,79.6072,-64.4663,-98.0552,2.287]

In [31]:
gradientObservations = transposedGradientObservations.transpose()
gradientObservations

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=30 columns=4 values=[-2.89841e-05,-57.5967,78.9632,-2.36888e-05,-47.0739,64.5367,-2.66114e-05,-52.8817,72.4991,-3.0868e-05,-61.3402,84.0955,-2.15444e-05,-42.8126,58.6947,-2.81713e-05,-55.9814,76.7487,-2.59698e-05,-51.6067,70.7511,-3.11457e-05,-61.8921,84.852,-2.9564e-05,-58.7489,80.5428,-2.92205e-05,-58.0665,79.6072,2.60114e-05,49.8552,-63.9447,2.12592e-05,40.7467,-52.2621,2.38821e-05,45.7739,-58.71,2.7702e-05,53.0956,-68.1008,1.93347e-05,37.0582,-47.5312,2.52819e-05,48.457,-62.1514,2.33062e-05,44.6703,-57.2945,2.79513e-05,53.5732,-68.7135,2.65318e-05,50.8525,-65.2239,2.62236e-05,50.2618,-64.4663,1.7956e-05,59.8535,-97.2619,1.46755e-05,48.9184,-79.4923,1.64861e-05,54.9537,-89.2998,1.91231e-05,63.7437,-103.584,1.3347e-05,44.4902,-72.2965,1.74525e-05,58.1749,-94.5343,1.60886e-05,53.6288,-87.1468,1.92952e-05,64.3172,-104.515,1.83153e-05,61.0508,-99.2076,1.81025e-05,60.3417,-98.0552,-4.188e-07,-1.396,2.2685,-3.42286e-07,-1.14095,1.85405,-3.84516e-07,-1.28172,2.0828,-4.4602e-07,-1.48673,2.41594,-3.11301e-07,-1.03767,1.68622,-4.07055e-07,-1.35685,2.20488,-3.75245e-07,-1.25082,2.03258,-4.50033e-07,-1.50011,2.43768,-4.27178e-07,-1.42393,2.31388,-4.22216e-07,-1.40739,2.287]

In [32]:
invLRJ = LR.solveLinearSystem(gradientObservations)
invLRJ

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=30 columns=4 values=[-144.921,-44.3051,60.7409,-118.444,-36.2107,49.6436,-133.057,-40.6782,55.7685,-154.34,-47.1848,64.6888,-107.722,-32.9328,45.1498,-140.856,-43.0626,59.0375,-129.849,-39.6974,54.4239,-155.728,-47.6093,65.2708,-147.82,-45.1915,61.956,-146.103,-44.6665,61.2363,130.057,38.3501,-49.1882,106.296,31.3436,-40.2016,119.41,35.2107,-45.1616,138.51,40.8427,-52.3853,96.6737,28.5063,-36.5625,126.41,37.2746,-47.8088,116.531,34.3618,-44.0727,139.756,41.2102,-52.8565,132.659,39.1173,-50.1722,131.118,38.6629,-49.5894,89.7802,46.0411,-74.8169,73.3775,37.6295,-61.1479,82.4306,42.2721,-68.6922,95.6156,49.0336,-79.6796,66.7352,34.2232,-55.6127,87.2624,44.75,-72.7187,80.4432,41.2529,-67.036,96.4758,49.4748,-80.3965,91.5763,46.9622,-76.3136,90.5125,46.4167,-75.4271,-2.094,-1.07385,1.745,-1.71143,-0.877656,1.42619,-1.92258,-0.985939,1.60215,-2.2301,-1.14364,1.85842,-1.55651,-0.798209,1.29709,-2.03527,-1.04373,1.69606,-1.87623,-0.962167,1.56352,-2.25016,-1.15393,1.87514,-2.13589,-1.09533,1.77991,-2.11108,-1.0826,1.75923]

In [33]:
llsqA = ot.Matrix(parameterDimension+size*outputDimension,parameterDimension)
llsqA[0:parameterDimension,0:parameterDimension] = invLB
for i in range(size):
    for j in range(outputDimension):
        for k in range(parameterDimension):
            llsqA[i*outputDimension + j + parameterDimension,k] = -invLRJ[i*outputDimension + j,k]
llsqA

class=Matrix implementation=class=MatrixImplementation name=Unnamed rows=34 columns=4 values=[7.14286,0,0,0,144.921,44.3051,-60.7409,118.444,36.2107,-49.6436,133.057,40.6782,-55.7685,154.34,47.1848,-64.6888,107.722,32.9328,-45.1498,140.856,43.0626,-59.0375,129.849,39.6974,-54.4239,155.728,47.6093,-65.2708,147.82,45.1915,-61.956,146.103,44.6665,-61.2363,0,8.33333,0,0,-130.057,-38.3501,49.1882,-106.296,-31.3436,40.2016,-119.41,-35.2107,45.1616,-138.51,-40.8427,52.3853,-96.6737,-28.5063,36.5625,-126.41,-37.2746,47.8088,-116.531,-34.3618,44.0727,-139.756,-41.2102,52.8565,-132.659,-39.1173,50.1722,-131.118,-38.6629,49.5894,0,0,14.2857,0,-89.7802,-46.0411,74.8169,-73.3775,-37.6295,61.1479,-82.4306,-42.2721,68.6922,-95.6156,-49.0336,79.6796,-66.7352,-34.2232,55.6127,-87.2624,-44.75,72.7187,-80.4432,-41.2529,67.036,-96.4758,-49.4748,80.3965,-91.5763,-46.9622,76.3136,-90.5125,-46.4167,75.4271,0,0,0,50,2.094,1.07385,-1.745,1.71143,0.877656,-1.42619,1.92258,0.985939,-1.60215,2.2301,1.14364,-1.85842,1.55651,0.798209,-1.29709,2.03527,1.04373,-1.69606,1.87623,0.962167,-1.56352,2.25016,1.15393,-1.87514,2.13589,1.09533,-1.77991,2.11108,1.0826,-1.75923]

In [34]:
invLRz = LR.solveLinearSystem(deltay)
invLRz

class=Point name=Unnamed dimension=30 values=[21.0534,2.81188,0.0412909,17.7673,2.17206,-0.800396,19.3025,0.997383,-1.14283,22.9844,4.37497,-0.953322,16.6634,2.00935,-2.4379,20.1177,2.37733,-1.07373,18.7389,2.63422,-1.50666,23.0396,4.01622,-0.374103,21.5103,1.12114,-0.405059,20.46,3.29319,-2.07798]

In [35]:
ybar = ot.Point(parameterDimension+size*outputDimension)
for i in range(size):
    for j in range(outputDimension):
        ybar[i*outputDimension + j + parameterDimension] = -invLRz[i*outputDimension + j]
ybar

class=Point name=Unnamed dimension=34 values=[0,0,0,0,-21.0534,-2.81188,-0.0412909,-17.7673,-2.17206,0.800396,-19.3025,-0.997383,1.14283,-22.9844,-4.37497,0.953322,-16.6634,-2.00935,2.4379,-20.1177,-2.37733,1.07373,-18.7389,-2.63422,1.50666,-23.0396,-4.01622,0.374103,-21.5103,-1.12114,0.405059,-20.46,-3.29319,2.07798]

In [36]:
method = ot.SVDMethod(llsqA)
method

class=SVDMethod

In [37]:
deltaTheta = method.solve(ybar)
deltaTheta

class=Point name=Unnamed dimension=4 values=[-0.12085,0.154987,-0.184491,0.000351264]

In [38]:
thetaStar = deltaTheta + candidate
thetaStar

class=Point name=Unnamed dimension=4 values=[1.27915,1.35499,0.515509,0.200351]

In [39]:
method.getGramInverse()

class=CovarianceMatrix dimension=4 implementation=class=MatrixImplementation name=Unnamed rows=4 columns=4 values=[0.00720375,0.00733857,0.000981353,-1.86847e-06,0.00733857,0.0075243,0.000947296,-1.80362e-06,0.000981353,0.000947296,0.000197625,8.95317e-06,-1.86847e-06,-1.80362e-06,8.95317e-06,0.000399983]